## Handling form errors

In chapter 13, we created a register function in actions.js, where we checked among the others if the passwords are matching.

In [ ]:
## actions.js

export const registerNewUser = async (formData) => {
    const {username, email, password, img, passwordRepeat} = Object.fromEntries(formData)

    if (password !== passwordRepeat) {
        return {error: "Passwords do not match!"}
    }

#...

### Important
###### Please notice that we return an error message, not throwing new error. If we throw an error Next.js will redirect us to an error page, and this is something we want to prevent.

We will use useStateHook. This hook allows you to update state based on the result of a form action.
As you remember if we use hooks in our components, it must be a "client" component, and our register page is a "server" component.

In this situation we have to create another component and move our form to it first. 
Than we have to import useFormState hook from "react-dom" and use a state variable

In [ ]:
"use client"  ## it must be a client component

import { registerNewUser } from "@/lib/actions"
import styles from "./registerForm.module.css"
import { useFormState } from "react-dom"   ## import useFormState from react-dom
import { useRouter } from "next/navigation" ## import useRouter to redirect to login page after successful registration
import { useEffect } from "react"
import Link from "next/link"

const RegisterForm = () => {

  ## utilizing useFormState - we pass function registerNewUser and a initial state which is undefined
  const [state, formAction] = useFormState(registerNewUser, undefined); 

  ## initialize router
  const router = useRouter();
  
  ## if state have success property equal to true, it triggers router and redirect to login page.
  ## "success" if passed from actions.js (registerNewUser function in Chapter 13 of Next.js notes)
  useEffect(() => {
    state?.success && router.push("/login")
  }, [state?.success, router])

  return (
    <form className={styles.form} action={formAction}> ## as an action we use formAction function.
        <input type="text" placeholder="username" name="username"/>
        <input type="text" placeholder="email" name="email"/>
        <input type="password" placeholder="password" name="password"/>
        <input type="password" placeholder="password again" name="passwordRepeat"/>
        <button>Register</button>
        {state?.error}
        <Link href='/login' className={styles.link}>
          Have an account? <b>Login</b>
        </Link>
      </form>
  )
}

export default RegisterForm

##### Important: We have to add a new parameter ("prevState") to registerNewUser function in action.js

In [ ]:
## action.js

export const registerNewUser = async (previousState, formData) => { ## prevState is added as a parameter
    const {username, email, password, img, passwordRepeat} = Object.fromEntries(formData)
#...

###### Login page

For the login page we have to do the same. Let's first adjust the LoginForm component.

In [ ]:
"use client" ## added use client

import styles from "./loginForm.module.css";
import { login } from "@/lib/actions";
import { useFormState } from "react-dom"   ## import useFormState from react-dom
import { useRouter } from "next/navigation" ## import useRouter to redirect to login page after successful registration
import { useEffect } from "react"
import Link from "next/link";


const LoginForm = () => {

  ## utilizing useFormState - we pass function registerNewUser and a initial state which is undefined
  const [state, formAction] = useFormState(registerNewUser, undefined); 

  ## initialize router
  const router = useRouter();

  return (
    <div>
      <form className={styles.form} action={formAction}>
          <input type='text' placeholder='login' name='username' />
          <input type='password' placeholder='password' name='password' />
          <button>Login</button> 
          <Link href="/register" className={styles.link}>
        {"Don't Have an account?"} <b>Register</b>
      </Link>
        </form>
    </div>
  )
}

export default LoginForm

Now we have to modify our login function in actions.js

In [ ]:
## actions.js

export const login = async (prevState, formData) => {
    const {username, password} = Object.fromEntries(formData)

    try {
        await signIn("credentials", {username, password});
    } catch (error) {
        console.log(error) 
        ## if the credentials are wrong an error will be thrown. If you conosle log error.message you will se a description
        ## of this error. Now, we can conditionally check what that message includes. If it includes a specific string, in this
        ## example "credentialssignin", we know it's a wrong credentials error.
        if (error.message.includes("credentialssignin")) {
            return {error: "Wrong username or password!"}
        }
        ## in the previous version we had "return {error: "Something went wrong"}"
        ## Now we have changed it to "throw error" Why? Because next-auth after successful authentication redirects to home page
        ## using redicrect() method, and that method intentionally throws an NEXT_REDIRECT error.
        ## This change in code will prevent that.
        throw error
    }
}